# 最初から機械学習でやってみる

In [4]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 7.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import json
import time
import glob
from typing import TypeVar, Type, List, Optional
from dataclasses import dataclass
from pydantic import BaseModel
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from openai import AzureOpenAI
from tqdm import tqdm
from typing import List, Dict, Optional, Any, Tuple

In [8]:
csv_files = glob.glob('*.csv')
print(csv_files)

['extracted_500_first_20241130_検索結果1204.csv', 'pubmed_true_data.csv']


In [9]:
df = pd.read_csv(csv_files[0])
# Assuming your dataframe is named 'df'
df['included'] = df['notes'].astype(str).str.contains('Included')

print(df['included'].value_counts())
print(f'不一致の数: {df[df["included"] == True]["notes"].str.contains("Excluded").sum()}')
#予測の元となる変数

import json

def create_json_text(row):
    return json.dumps({
        'title': str(row['title']),
        'abstract': str(row['abstract'])
    })

df['tiab'] = df.apply(create_json_text, axis=1)

included
False    493
True       7
Name: count, dtype: int64
不一致の数: 0


In [5]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
import optuna
from lightgbm import LGBMClassifier

def prepare_text_data(df, text_column='tiab', label_column='included'):
    """テキストデータの前処理とTF-IDF変換"""
    texts = df[text_column].fillna('')

    vectorizer = TfidfVectorizer(
        max_features=10000,
        min_df=2,
        max_df=0.95,
        ngram_range=(1, 2)
    )

    X_vec = vectorizer.fit_transform(texts)
    y = df[label_column]

    class_weights = dict(zip(
        y.unique(),
        [1 / (len(y) * (y == label).mean()) for label in y.unique()]
    ))

    return X_vec, y, vectorizer, class_weights

def fbeta_score_custom(y_true, y_pred, beta=1):
    """カスタムF-betaスコアの計算"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    if precision == 0.0 and recall == 0.0:
        return 0.0
    fbeta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
    return fbeta

def calculate_detailed_metrics(y_true, y_pred):
    """詳細な評価指標の計算と表示"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0

    print("\n=== 詳細な評価指標 ===")
    print(f"感度 (Sensitivity/Recall): {sensitivity:.3f}")
    print(f"特異度 (Specificity): {specificity:.3f}")
    print(f"適合率 (Precision): {precision:.3f}")
    print(f"F1スコア: {f1:.3f}")

    print("\n=== 混同行列 ===")
    print("                  Predicted")
    print("                  Negative  Positive")
    print(f"Actual Negative    {tn:^8} {fp:^8}")
    print(f"      Positive    {fn:^8} {tp:^8}")

    return {
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision,
        'f1': f1,
        'confusion_matrix': {'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp}
    }

def evaluate_model(X_vec, y, params, threshold, class_weights, beta=2):
    """モデルの評価（交差検証）"""
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    all_predictions = []
    all_true_values = []

    for train_idx, val_idx in skf.split(X_vec, y):
        X_tr = X_vec[train_idx]
        X_val = X_vec[val_idx]
        y_tr = y.iloc[train_idx]
        y_val = y.iloc[val_idx]

        model = LGBMClassifier(
            **params,
            random_state=42,
            class_weight=class_weights
        )

        model.fit(X_tr, y_tr)
        y_prob = model.predict_proba(X_val)[:, 1]
        y_pred = (y_prob >= threshold).astype(int)

        score = fbeta_score_custom(y_val, y_pred, beta=beta)
        scores.append(score)

        all_predictions.extend(y_pred)
        all_true_values.extend(y_val)

    if len(scores) == 5:
        print("\n=== 交差検証全体の評価 ===")
        metrics = calculate_detailed_metrics(all_true_values, all_predictions)
        print("\nClassification Report:")
        print(classification_report(all_true_values, all_predictions))

    return np.mean(scores)

def objective(trial, X_vec, y, class_weights, beta=2):
    """Optunaの目的関数"""
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 8, 128),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "verbosity": -1
    }
    threshold = trial.suggest_float("threshold", 0.1, 0.9)

    score = evaluate_model(X_vec, y, params, threshold, class_weights, beta=beta)
    return score

def train_final_model(df, beta_value=2, n_trials=50):
    """完全なトレーニングパイプライン"""
    # データの準備
    X_vec, y, vectorizer, class_weights = prepare_text_data(df)

    print(f"データセットの形状: {X_vec.shape}")
    print(f"クラスの分布:\n{y.value_counts(normalize=True)}")
    print(f"クラスの重み: {class_weights}")

    # ハイパーパラメータの最適化
    study = optuna.create_study(direction="maximize")
    study.optimize(
        lambda trial: objective(trial, X_vec, y, class_weights, beta=beta_value),
        n_trials=n_trials
    )

    best_params = study.best_trial.params
    threshold = best_params.pop("threshold")

    print("\nBest parameters:", best_params)
    print("Best threshold:", threshold)

    final_score = evaluate_model(X_vec, y, best_params, threshold, class_weights, beta=beta_value)
    print(f"\nFinal Mean F{beta_value} Score:", final_score)

    # 最終モデルの学習
    final_model = LGBMClassifier(
        **best_params,
        random_state=42,
        class_weight=class_weights
    )
    final_model.fit(X_vec, y)

    # 最終評価
    print("\n=== 最終モデルの評価（全データ） ===")
    y_prob = final_model.predict_proba(X_vec)[:, 1]
    y_pred = (y_prob >= threshold).astype(int)
    final_metrics = calculate_detailed_metrics(y, y_pred)

    return final_model, vectorizer, threshold, best_params

def predict_new_text(text, model, vectorizer, threshold):
    """新しいテキストの予測"""
    X_new = vectorizer.transform([text])
    prob = model.predict_proba(X_new)[0, 1]
    prediction = prob >= threshold
    return prediction, prob



In [10]:
# 使用例
if __name__ == "__main__":
    # データの読み込み
    #df = pd.read_csv('your_data.csv')  # tiab列とincluded列を含むデータ

    # モデルのトレーニング
    model, vectorizer, threshold, best_params = train_final_model(
        df,
        beta_value=4,  # より高い値で再現率を重視
        n_trials=50
    )


[I 2024-12-07 06:39:34,639] A new study created in memory with name: no-name-791c9cec-4e66-48ea-b25e-7be700f78b6d


データセットの形状: (500, 10000)
クラスの分布:
included
False    0.986
True     0.014
Name: proportion, dtype: float64
クラスの重み: {np.False_: np.float64(0.002028397565922921), np.True_: np.float64(0.14285714285714285)}


[I 2024-12-07 06:39:35,456] Trial 0 finished with value: 0.0 and parameters: {'num_leaves': 90, 'learning_rate': 0.019446585785254194, 'n_estimators': 116, 'reg_lambda': 0.04347690116313084, 'min_child_samples': 52, 'threshold': 0.5241715322992514}. Best is trial 0 with value: 0.0.



=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 0.000
特異度 (Specificity): 0.959
適合率 (Precision): 0.000
F1スコア: 0.000

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative      473       20   
      Positive       7        0    

Classification Report:
              precision    recall  f1-score   support

       False       0.99      0.96      0.97       493
        True       0.00      0.00      0.00         7

    accuracy                           0.95       500
   macro avg       0.49      0.48      0.49       500
weighted avg       0.97      0.95      0.96       500



[I 2024-12-07 06:39:36,247] Trial 1 finished with value: 0.0 and parameters: {'num_leaves': 104, 'learning_rate': 0.14019770800548456, 'n_estimators': 217, 'reg_lambda': 0.04551234981347055, 'min_child_samples': 57, 'threshold': 0.18956718191486843}. Best is trial 0 with value: 0.0.


...


=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 1.000
特異度 (Specificity): 0.000
適合率 (Precision): 0.014
F1スコア: 0.028

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative       0       493   
      Positive       0        7    

Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       493
        True       0.01      1.00      0.03         7

    accuracy                           0.01       500
   macro avg       0.01      0.50      0.01       500
weighted avg       0.00      0.01      0.00       500



[I 2024-12-07 06:40:37,564] Trial 48 finished with value: 0.2644444444444444 and parameters: {'num_leaves': 90, 'learning_rate': 0.003627606694375339, 'n_estimators': 398, 'reg_lambda': 0.05397920294256693, 'min_child_samples': 35, 'threshold': 0.44880202746350556}. Best is trial 34 with value: 0.34689189189189185.



=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 0.429
特異度 (Specificity): 0.933
適合率 (Precision): 0.083
F1スコア: 0.140

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative      460       33   
      Positive       4        3    

Classification Report:
              precision    recall  f1-score   support

       False       0.99      0.93      0.96       493
        True       0.08      0.43      0.14         7

    accuracy                           0.93       500
   macro avg       0.54      0.68      0.55       500
weighted avg       0.98      0.93      0.95       500



c:\Users\youki\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\youki\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\youki\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo


=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 1.000
特異度 (Specificity): 0.000
適合率 (Precision): 0.014
F1スコア: 0.028

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative       0       493   
      Positive       0        7    

Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       493
        True       0.01      1.00      0.03         7

    accuracy                           0.01       500
   macro avg       0.01      0.50      0.01       500
weighted avg       0.00      0.01      0.00       500


Best parameters: {'num_leaves': 43, 'learning_rate': 0.00597124763190807, 'n_estimators': 325, 'reg_lambda': 0.005217512945281993, 'min_child_samples': 18}
Best threshold: 0.1922612484366543

=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 0.571
特異度 (Specificity): 0.895
適合率 (Precision): 0.071
F1スコア: 0.127

=== 混同行列 ===
                  Predicted
       

In [11]:
import joblib


In [12]:

# モデルと関連オブジェクトの保存
model_data = {
    'model': model,
    'vectorizer': vectorizer,
    'threshold': threshold,
    'best_params': best_params
}
joblib.dump(model_data, 'model_data.joblib')
print("\nモデルと関連オブジェクトを 'model_data.joblib' に保存しました。")


モデルと関連オブジェクトを 'model_data.joblib' に保存しました。


In [20]:
# 保存したモデルデータをロード
model_data = joblib.load('model_data.joblib')

# 各オブジェクトの取り出し
loaded_model = model_data['model']
loaded_vectorizer = model_data['vectorizer']
loaded_threshold = model_data['threshold']
loaded_best_params = model_data['best_params']

print("モデルと関連オブジェクトをロードしました。")


モデルと関連オブジェクトをロードしました。


In [14]:
df2 = pd.read_csv(csv_files[1])

In [18]:
df2.columns = ["PMID", "title", "abstract"]

In [19]:
df2["tiab"] = df2.apply(create_json_text, axis=1)

In [22]:
df2.shape

(10, 4)

In [21]:
# テキストデータの前処理とベクトル化
X_new = loaded_vectorizer.transform(df2["tiab"].fillna(''))

print(f"新しいデータの形状: {X_new.shape}")

新しいデータの形状: (10, 10000)


In [23]:
# 予測確率の取得
y_prob_new = loaded_model.predict_proba(X_new)[:, 1]

# 閾値を適用してクラスラベルを決定
y_pred_new = (y_prob_new >= loaded_threshold).astype(int)

# 予測結果をデータフレームに追加
df2['prediction'] = y_pred_new
df2['probability'] = y_prob_new

print("予測を実行し、結果をデータフレームに追加しました。")


予測を実行し、結果をデータフレームに追加しました。


In [24]:
# 予測結果の表示
print("\n=== 予測結果 ===")
print(df2[['tiab', 'prediction', 'probability']])

# クラス分布の確認
print("\nクラスの分布:")
print(df2['prediction'].value_counts())

# 確率の統計情報
print("\n予測確率の統計情報:")
print(df2['probability'].describe())



=== 予測結果 ===
                                                tiab  prediction  probability
0  {"title": "A multicenter, randomized, controll...           1     0.312545
1  {"title": "Transfusion requirements after head...           1     0.211932
2  {"title": "Restrictive vs Liberal Transfusion ...           1     0.206408
3  {"title": "Liberal or Restrictive Transfusion ...           1     0.290118
4  {"title": "Effect of Transfusion on Mortality ...           1     0.233342
5  {"title": "Transfusion-related risk of seconda...           1     0.351916
6  {"title": "Impact of Blood Product Transfusion...           1     0.269007
7  {"title": "Transfusion practice and blood stre...           1     0.291831
8  {"title": "Anemia, transfusions and hospital o...           1     0.298781
9  {"title": "Transfusion of platelets, but not o...           0     0.084553

クラスの分布:
prediction
1    9
0    1
Name: count, dtype: int64

予測確率の統計情報:
count    10.000000
mean      0.255043
std       0.075384